In [6]:
from database.sim import Sim
import pandas as pd
from tqdm import tqdm
from datetime import datetime, timedelta, timezone
import matplotlib.pyplot as plt
import math

In [13]:
sim = Sim("sim")
start = datetime(2018,1,1).replace(tzinfo=timezone.utc)
end = datetime(2021,1,1).replace(tzinfo=timezone.utc)

In [9]:
sim.connect()
data = sim.retrieve_data("sim")
sim.close()

In [10]:
data

,_id,date,adjclose,quarter,year,predicted,ticker,score
0,5ff3366bbdf5d91d77e57056,2017-06-30,187.130188,1,2017,166.741769,MMM,0.755751
1,5ff3366bbdf5d91d77e57057,2017-06-30,187.130188,1,2017,166.741769,MMM,0.755751
2,5ff3366bbdf5d91d77e57058,2017-07-03,188.604279,1,2017,166.741769,MMM,0.755751
3,5ff3366bbdf5d91d77e57059,2017-07-03,188.604279,1,2017,166.741769,MMM,0.755751
4,5ff3366bbdf5d91d77e5705a,2017-07-05,188.541351,1,2017,166.741769,MMM,0.755751
...,...,...,...,...,...,...,...,...
616592,5ff344c3bdf5d91d77eede83,2020-09-23,94.341675,1,2020,104.457414,ETR,-0.422862
616593,5ff344c3bdf5d91d77eede84,2020-09-24,94.946365,1,2020,104.457414,ETR,-0.422862
616594,5ff344c3bdf5d91d77eede85,2020-09-24,94.946365,1,2020,104.457414,ETR,-0.422862
616595,5ff344c3bdf5d91d77eede86,2020-09-25,96.135933,1,2020,104.457414,ETR,-0.422862


In [11]:
data["date"] = [x.replace(tzinfo=timezone.utc) for x in data["date"]]
data["predicted_delta"] = (data["predicted"] - data["adjclose"]) / data["adjclose"]
filtered_sim = data[data["predicted_delta"] <= 1]

In [12]:
filtered_sim

,_id,date,adjclose,quarter,year,predicted,ticker,score,predicted_delta
0,5ff3366bbdf5d91d77e57056,2017-06-30 00:00:00+00:00,187.130188,1,2017,166.741769,MMM,0.755751,-0.108953
1,5ff3366bbdf5d91d77e57057,2017-06-30 00:00:00+00:00,187.130188,1,2017,166.741769,MMM,0.755751,-0.108953
2,5ff3366bbdf5d91d77e57058,2017-07-03 00:00:00+00:00,188.604279,1,2017,166.741769,MMM,0.755751,-0.115917
3,5ff3366bbdf5d91d77e57059,2017-07-03 00:00:00+00:00,188.604279,1,2017,166.741769,MMM,0.755751,-0.115917
4,5ff3366bbdf5d91d77e5705a,2017-07-05 00:00:00+00:00,188.541351,1,2017,166.741769,MMM,0.755751,-0.115622
...,...,...,...,...,...,...,...,...,...
616592,5ff344c3bdf5d91d77eede83,2020-09-23 00:00:00+00:00,94.341675,1,2020,104.457414,ETR,-0.422862,0.107224
616593,5ff344c3bdf5d91d77eede84,2020-09-24 00:00:00+00:00,94.946365,1,2020,104.457414,ETR,-0.422862,0.100173
616594,5ff344c3bdf5d91d77eede85,2020-09-24 00:00:00+00:00,94.946365,1,2020,104.457414,ETR,-0.422862,0.100173
616595,5ff344c3bdf5d91d77eede86,2020-09-25 00:00:00+00:00,96.135933,1,2020,104.457414,ETR,-0.422862,0.086560


In [14]:
def backtest(sim,delta_req,seats):
    trades = []
    blacklist = pd.DataFrame([{"ticker":"ZZZZZ","start":datetime(2016,4,1).replace(tzinfo=timezone.utc)
                               ,"end":datetime(2016,4,14).replace(tzinfo=timezone.utc)}])
    for i in range(1,seats):
        date = start
        while date < end:
            if date.weekday() > 4:
                date = date + timedelta(days=7-date.weekday())
            blacklist_tickers = blacklist[(blacklist["start"] <= date) & (blacklist["end"] >= date)]["ticker"]
            todays_sim = sim[(~sim["ticker"].isin(blacklist_tickers)) \
                             & (sim["date"] == date) & (sim["predicted_delta"] >= delta_req)]
            if todays_sim.index.size >= 1:
                offerings = todays_sim.sort_values("predicted_delta",ascending=False)
                if offerings.index.size < 1:
                    date = date + timedelta(days=1)
                else:
                    for offering in range(offerings.index.size):
                        try:
                            trade = offerings.iloc[offering]
                            sell_date = trade["date"] + timedelta(days=1)
                            sell_trades = sim[(sim["date"] >= sell_date)  & (sim["date"] <= sell_date + timedelta(days=90)) & (sim["ticker"] == trade["ticker"])]
                            if sell_trades.index.size < 1:
                                if offering == offerings.index.size - 1:
                                    date = date + timedelta(days=1)
                                    break
                                else:
                                    continue
                            else:
                                sell_trades["delta"] = (sell_trades["adjclose"] - trade["adjclose"]) / trade["adjclose"]
                                positive_trades = sell_trades[sell_trades["delta"] >= delta_req]
                                if positive_trades.index.size < 1:
                                    sell_trade = sell_trades.sort_values("delta",ascending=False).iloc[0]
                                else:
                                    sell_trade = positive_trades.iloc[0]
                                trade["sell_price"] = sell_trade["adjclose"]
                                trade["sell_date"] = sell_trade["date"]
                                trade["sell_delta"] = float(sell_trade["delta"])
                                trade["seat"] = i
                                trade["predicted_delta"] = sell_trade["predicted_delta"]
                                blacklist = blacklist.append([{"ticker":trade["ticker"],"start":trade["date"],"end":sell_trade["date"]}])
                                trades.append(trade)
                                date = sell_trade["date"] + timedelta(days=1)
                                break
                        except Exception as e:
                            print("packaging",i,str(e))
                            date = date + timedelta(days=1)
            else:
                date = date + timedelta(days=1)
    return trades

In [18]:
sim.connect()
seats = 21
epoch = 0
sim.drop_table("epochs")
for i in tqdm(range(5,25,5)):
    try:
        dr = i/100
        trades = backtest(filtered_sim,dr,seats)
        epoch_dict = {"epoch":epoch,"delta_req":i}
        try:
            sim.drop_table("trades_{}".format(epoch))
        except:
            print("no table")
        if len(trades) > 1:
            sim.store_data("epochs",pd.DataFrame([epoch_dict]))
            sim.store_data("trades_{}".format(epoch),pd.DataFrame(trades))
        epoch += 1
    except Exception as e:
        print(str(e))
sim.close()

  0%|                                                                                                                                   | 0/4 [00:00<?, ?it/s]<ipython-input-14-6ac78b0f03b5>:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sell_trades["delta"] = (sell_trades["adjclose"] - trade["adjclose"]) / trade["adjclose"]
<ipython-input-14-6ac78b0f03b5>:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trade["sell_price"] = sell_trade["adjclose"]
c:\users\eric\appdata\local\programs\python\python38\lib\site-packages\pandas\core\indexing.py:671: SettingWithCo

In [19]:
pd.DataFrame(trades)

,_id,date,adjclose,quarter,year,predicted,ticker,score,predicted_delta,sell_price,sell_date,sell_delta,seat
14262,5ff336c9bdf5d91d77e5a860,2017-07-03 00:00:00+00:00,9.654622,1,2017,17.354471,F,-0.723295,0.070313,10.226938,2017-10-02 00:00:00+00:00,0.059279,1
34738,5ff33758bdf5d91d77e5f8f6,2017-10-03 00:00:00+00:00,97.674164,2,2017,193.120434,AAP,-0.630602,0.437437,104.754112,2018-01-02 00:00:00+00:00,0.072485,1
69640,5ff33841bdf5d91d77e68237,2018-01-03 00:00:00+00:00,64.036087,3,2017,108.049030,ATVI,-403.106865,0.408828,76.694267,2018-03-09 00:00:00+00:00,0.197673,1
74142,5ff3385cbdf5d91d77e693d1,2018-03-12 00:00:00+00:00,33.042454,3,2017,54.913019,APA,0.821621,0.074009,39.975681,2018-04-18 00:00:00+00:00,0.209828,1
128394,5ff339a9bdf5d91d77e76839,2018-04-19 00:00:00+00:00,56.442131,4,2017,85.581674,CAH,0.861625,0.435414,59.621590,2018-04-27 00:00:00+00:00,0.056331,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
586226,5ff34419bdf5d91d77ee67b3,2020-06-24 00:00:00+00:00,70.650002,4,2019,127.010831,RL,0.488104,0.806184,70.320000,2020-06-25 00:00:00+00:00,-0.004671,20
544587,5ff34328bdf5d91d77edc4d4,2020-06-26 00:00:00+00:00,34.197838,4,2019,52.975816,AFL,0.840413,-0.209228,37.614780,2020-09-04 00:00:00+00:00,0.099917,20
609009,5ff34499bdf5d91d77eec0d7,2020-09-08 00:00:00+00:00,19.254566,1,2020,24.306491,CNP,0.899353,0.238714,19.622360,2020-09-16 00:00:00+00:00,0.019102,20
606633,5ff3448cbdf5d91d77eeb788,2020-09-21 00:00:00+00:00,1652.390015,1,2020,2007.470933,BKNG,0.417843,0.199206,1674.000000,2020-09-22 00:00:00+00:00,0.013078,20
